## Setup

In [1]:
import os
from sys import path as sys_path
from shutil import rmtree

sys_path.insert(0, '.') # Import repo's evadb

from evadb import connect
from evadb.mojo import MOJO_BUILTINS_PATH
from evadb.functions.function_bootstrap_queries import Similarity_function_query, Text_feat_function_query
if os.path.exists("evadb_data"):
    rmtree("evadb_data", ignore_errors=True)
print("⏳ Establishing evadb connection...")
cursor = connect().cursor()
cursor.query(Similarity_function_query).execute()

⏳ Establishing evadb connection...


## Benchmark: Load and Setup time: Python vs Mojo

### Python

##### Note: If it's taking too long or it's showing output of a download, it's possible that it's downloading a model for the first time. Restart the notebook after it finishes

In [2]:
cursor.query(Text_feat_function_query).execute()
pythonQ = cursor.query("SELECT Similarity(SentenceFeatureExtractor('hi').features, SentenceFeatureExtractor('bye').features)")

/home/ileva3/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Mojo

In [3]:
cursor.query(f"CREATE OR REPLACE FUNCTION SentenceTransformerFeatureExtractor IMPL '{MOJO_BUILTINS_PATH}'").execute()
mojoQ = cursor.query("SELECT Similarity(SentenceTransformerFeatureExtractor('hi').features, SentenceTransformerFeatureExtractor('bye').features)")

## Benchmark: Function Execution Time: Python vs Mojo

### Python

In [4]:
%%timeit
pythonQ.df()

655 ms ± 9.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Mojo

In [5]:
%%timeit
mojoQ.df()

128 ms ± 2.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Mojo Features: multiple functions per binary, support for non-builtin functions

In [6]:
cursor.query(f"CREATE OR REPLACE FUNCTION CustomSentenceTransformerFeatureExtractor1 IMPL './mojo-demo/CustomSourceSTFES'").execute()

In [7]:
cursor.query(f"CREATE OR REPLACE FUNCTION CustomSentenceTransformerFeatureExtractor2 IMPL './mojo-demo/CustomSourceSTFES'").execute()

In [8]:
cursor.query(f"SELECT Similarity(CustomSentenceTransformerFeatureExtractor1('hi').features, CustomSentenceTransformerFeatureExtractor2('bye').features)").df()

,distance
0,1.262829


## Stop all Mojo processes (done by atexit in regular python scripts)

In [9]:
from evadb.mojo import MojoController
MojoController.stop_all()

## Stop EvaDB

In [10]:
cursor.close()
if os.path.exists("evadb_data"):
    rmtree("evadb_data", ignore_errors=True)

# Benchmark Conclusion